<a href="https://colab.research.google.com/github/Masum06/TinyAgent/blob/main/TinyAgent_Colab_Noebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [ ]:
_set_env("OPENAI_API_KEY")

OPENAI_API_KEY: ··········


In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
!pip install -q tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.1 MB/s eta 0:00:00


# TinyAgent

In [ ]:
import json
import tiktoken

class TinyAgent:
  def __init__(self, name="", system=""):
    self.name = name
    self.system = system
    self.messages = []
    self.temperature = 1
    self.max_tokens = 2048
    self.model = "gpt-4.1"
    if name:
      self.add_system_message(f"Your name is {name}.")
    if system:
      self.add_system_message(system)

  def token_count(self, string):
    encoding = tiktoken.encoding_for_model("gpt-4o") # 4.1 not in tiktoken yet
    num_tokens = len(encoding.encode(string))
    return num_tokens

  def prompt_token_count(self):
    prompt = ""
    for message in self.messages:
      prompt += message["content"]
    return self.token_count(prompt)

  def add_message(self,message_type, message):
    self.messages.append({"role": message_type, "content":message})

  def add_system_message(self, message):
    self.add_message("system", message)

  def add_user_message(self, message):
    self.add_message("user", message)

  def add_instruction(self, instruction):
    self.add_system_message(f"Follow this instruction: \n{instruction}\n\n")

  def add_example(self, input, output):
    self.add_system_message(f"Example Input: {input}\nExample Output: {output}\n\n")

  def add_data(self, data):
    self.add_user_message(f"Data: {data}\n\n")

  def set_temperature(self, temperature):
    self.temperature = temperature

  def set_max_tokens(self, max_tokens):
    self.max_tokens = max_tokens

  def set_model(self, model):
    self.model = model

  def call(self, prompt="", response_type="text"):
    if prompt:
      self.add_user_message(prompt)
    response = client.chat.completions.create(
      model=self.model,
      messages=self.messages,
      temperature=self.temperature,
      max_tokens=max(self.max_tokens, int(self.prompt_token_count()*2)),
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      response_format={
        "type": response_type # "text" or "json_object"
      }
    )
    reply = response.choices[0].message.content
    self.add_message("assistant", reply)
    return reply

  def load_json(self,s):
    try:
        # Attempt to parse the matched JSON
        return json.loads(s)
    except json.JSONDecodeError:
        # Return None if JSON parsing fails
        return None

  def call_json(self, prompt=""):
    prompt += "\n\nOutput must be JSON format.\n\n"
    reply = self.call(prompt, response_type="json_object")
    try:
      reply_json = self.load_json(reply)
      if reply_json[-1] != "}":
        raise Exception("Incomplete JSON")
    except:
      reply_json = None
      print("Error parsing JSON")

    return reply_json

In [ ]:
tiny_agent = TinyAgent("tiny_agent 🤖", "You perform math.")

In [ ]:
tiny_agent.messages

[{'role': 'system', 'content': 'Your name is tiny_agent 🤖.'},
 {'role': 'system', 'content': 'You perform math.'}]

## Unstructured Call

In [ ]:
tiny_agent.call("What's your name?")

'My name is tiny_agent 🤖. How can I help you with math today?'

In [ ]:
tiny_agent.messages

[{'role': 'system', 'content': 'Your name is tiny_agent 🤖.'},
 {'role': 'system', 'content': 'You perform math.'},
 {'role': 'user', 'content': "What's your name?"},
 {'role': 'assistant',
  'content': 'My name is tiny_agent 🤖. How can I help you with math today?'}]

## Structured Call

In [ ]:
result = tiny_agent.call_json("Give me a list of popular math equations in JSON format.")
result

{'popular_math_equations': [{'name': 'Pythagorean Theorem',
   'equation': 'a^2 + b^2 = c^2'},
  {'name': 'Quadratic Formula', 'equation': 'x = (-b ± √(b^2 - 4ac)) / (2a)'},
  {'name': "Euler's Formula (Complex Analysis)",
   'equation': 'e^{ix} = cos(x) + i·sin(x)'},
  {'name': 'Area of a Circle', 'equation': 'A = πr^2'},
  {'name': 'Slope of a Line', 'equation': 'm = (y_2 - y_1) / (x_2 - x_1)'},
  {'name': "Newton's Second Law", 'equation': 'F = ma'},
  {'name': 'Logarithm Definition', 'equation': 'log_b(a) = c ↔ b^c = a'},
  {'name': 'Binomial Theorem',
   'equation': '(a + b)^n = Σ_{k=0}^n (n choose k) a^{n−k} b^k'},
  {'name': 'Derivative Definition',
   'equation': "f'(x) = lim_{h→0} [(f(x+h) - f(x)) / h]"},
  {'name': 'Sum of Arithmetic Series', 'equation': 'S_n = n/2 · (a_1 + a_n)'}]}

In [ ]:
result['popular_math_equations'][0]

{'name': 'Pythagorean Theorem', 'equation': 'a^2 + b^2 = c^2'}